In [20]:
import networkx as nx
import pandas as pd
import json

In [15]:
edge_file = "../../data/knnsplit/graph_toronto_8.csv"
edges = pd.read_csv(edge_file, ' ', header = 0)
G = nx.from_pandas_edgelist(edges, source = 'r1', target='r2')

In [16]:
print nx.number_connected_components(G)

6


In [22]:
mapping = {}
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        mapping[node] = i

out_file = "../../data/knnsplit/community_toronto_8.csv"

with open(out_file, "w+") as f:
    json.dump(mapping, f)